## Task 3: Deploy a Model in Amazon SageMaker

In this task, you deploy the trained machine learning model using Amazon SageMaker to get predictions. You first export the model artifact to an Amazon S3 bucket. You then create a Sagemaker Model object from model artifact. Next, you create a batch transform job by specifying the Sagemaker Model, the S3 bucket location of the data to be transformed and the S3 bucket location to store the results. You then start the batch transform job and monitor its progress. Finally, once the batch transform job is complete, you retrieve the results from the specified S3 bucket. The results will contain the inferences made by the trained model on the input data.

**Note:** Solutions are available in the <a href="#appendix" target="_self">**Appendix**</a> section at the bottom of the notebook if you experience any issues completing the challenges.

### Task 3.1 Set up the environment

This basic setup code has been included to help you get started. Read and run these cells first to get packages installed and variables created.

In [2]:
#import libraries and set variable values

import sys
import boto3
import os
import sagemaker
import time

sess = sagemaker.Session()

bucket = sess.default_bucket()
prefix = "lab3/batch-transform"
bucket_path = "s3://{}".format(bucket)

role = sagemaker.get_execution_role()

region = boto3.Session().region_name


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


### Task 3.2 Create a model

In this task, you take the pretrained model and upload it to S3. You then create a model in SageMaker. The model will later be deployed using the SageMaker Batch Transform job.

#### Requirements and Configuration

- Use **S3.Client.upload_file** API to upload the model artifact and test data to an S3 bucket.

Refer to [Upload a file to an S3 object](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3/client/upload_file.html#upload-file) for more information about using the **upload_file** method.

In [4]:
# Upload the model to the Amazon S3 bucket
from sagemaker.session import Session
from sagemaker.image_uris import retrieve

sagemaker = boto3.client('sagemaker')
s3_client = boto3.client("s3")

#add_upload-model_code_here
s3_client.upload_file(Filename="model.tar.gz", Bucket=bucket, Key=f"{prefix}/models/model.tar.gz") 
model_path = f"s3://{bucket}/{prefix}/models/model.tar.gz"
print(model_path)

#add_upload-dataset_code_here
s3_client.upload_file(Filename="test_data.csv", Bucket=bucket, Key=f"{prefix}/test_data.csv", ExtraArgs={"ContentType": "text/csv;charset=utf-8"})
batch_path = f"s3://{bucket}/{prefix}/test_data.csv"
print(batch_path)

s3://sagemaker-us-east-1-333900834230/lab3/batch-transform/models/model.tar.gz
s3://sagemaker-us-east-1-333900834230/lab3/batch-transform/test_data.csv


**Note:** For detailed code on how to upload the model artifact and test data to an S3 bucket, refer to <a href="#task3-2-1" target="_self">**Upload model and test data to S3 (Task 3.2)**</a> in the *Appendix* section. 

<a id="task3-2-continue"></a>

Next, create a model object in SageMaker from model artifacts above.

#### Requirements and Configuration

- Use **SageMaker.Client.create_model** API to create a model in SageMaker.
- In the Request, use the following options:
    - ModelName='telemarketing-model'
    - ExecutionRoleArn=role (The IAM role create earlier that SageMaker uses)
    - For the primary container, use the following options:
        - Image= XGBoost container image URI (retrieve(framework='xgboost', region=boto3.Session().region_name, version='1.5-1'))
        - ModelDataUrl=model_path (The S3 path where the model artifacts are uploaded earlier) 

Refer to [Creating a model in SageMaker](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker/client/create_model.html) for more information about using the **SageMaker.Client.create_model** method.

In [5]:
sagemaker = boto3.client('sagemaker')

#add_create-model_code_here

# Retrieve the container imagez
image = retrieve(framework='xgboost', region=boto3.Session().region_name, version='1.5-1')

primary_container = {"Image": image, "ModelDataUrl": model_path}

# Set up the model
create_model_response = sagemaker.create_model(
    ModelName='telemarketing-model', ExecutionRoleArn=role, PrimaryContainer=primary_container
)

print(create_model_response["ModelArn"])

arn:aws:sagemaker:us-east-1:333900834230:model/telemarketing-model


**Note:** For detailed code on how to create a model, refer to <a href="#task3-2-2" target="_self">**Create a model in SageMaker (Task 3.2)**</a> in the *Appendix* section. 

<a id="task3-3"></a>

### Task 3.3 Create a batch transformer

In this task, you create a batch transformer.

Refer to [Using Transformer API](https://sagemaker.readthedocs.io/en/stable/api/inference/transformer.html#sagemaker.transformer.Transformer) for more information about using the **Transfomer** API.

#### Requirements and Configuration

- Use **sagemaker.transformer.Transformer** API to initialize a transformer in SageMaker.
- In the Parameters section, use the following options:
    - model_name="telemarketing-model"
    - instance_type="ml.m4.xlarge"
    - instance_count=1
    - assemble_with="Line"
    - strategy="MultiRecord"
    - accept="text/csv"
    - output_path="s3://{}/{}/batch-transform/test".format(bucket, prefix)

In [6]:
from sagemaker.transformer import Transformer

#add_create-batch-transformer_code_here
transformer = Transformer(
    model_name="telemarketing-model",
    instance_type="ml.m4.xlarge",
    instance_count=1,
    assemble_with="Line",
    strategy="MultiRecord",
    accept="text/csv",
    output_path="s3://{}/{}/batch-transform/test".format(bucket, prefix)
)

**Note:** For detailed code on how to create a Transformer object, refer to <a href="#task3-3-1" target="_self">**Create a Batch Transformer Object (Task 3.3)**</a> in the *Appendix* section. 

<a id="task3-4"></a>

### Task 3.4: Run the batch transform job

In this task, you run the transform job. You will pass the training data in to get predictions from the batch transformer.

Refer to [Using Transform API](https://sagemaker.readthedocs.io/en/stable/api/inference/transformer.html#sagemaker.transformer.Transformer.transform) for more information about using the **Transformer.transform** API.

#### Requirements and Configuration

- Use **Transformer.transform** API to initialize a transformer in SageMaker.
- In the Parameters section, use the following options:
    - batch_path (Input data location that Amazon S3 created earlier)
    - content_type="text/csv"
    - split_type="Line"
    - join_source="Input"

In [7]:
#run-batch-transform-job
transformer.transform(batch_path, content_type="text/csv", split_type="Line", join_source="Input")

transformer.wait()

INFO:sagemaker:Creating transform job with name: sagemaker-xgboost-2024-10-26-14-27-52-112


..........................................
...

**Note:** For detailed code on how to run a transform job, refer to <a href="#task3-4-1" target="_self">**Run a batch transform job (Task 3.4)**</a> in the *Appendix* section. 

<a id="task3-5"></a>

### Task 3.5: Verify the output stored in an S3 bucket

In this task, you pull the the output of the Batch Transform job in S3. It should show the list of predictions.

In [8]:
!aws s3 cp --recursive $transformer.output_path ./
!head test_data.csv.out

download: s3://sagemaker-us-east-1-333900834230/lab3/batch-transform/batch-transform/test/test_data.csv.out to ./test_data.csv.out
56,1,966,11,5,505,2,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,1,0,1,0,1,0,0.2391170859336853
50,1,2564,21,11,124,4,113,8,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,1,0,0.0140683026984334
34,1,-28,16,7,574,1,-1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,1,0,1,0,0.22969332337379456
36,2,91,7,5,371,2,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,1,0,0.012043386697769165
46,0,2215,6,6,222,2,-1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,1,0,0.014749388210475445
55,1,154,11,5,70,2,367,3,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,1,1,0,0.002062760293483734
37,0,681,5,2,187,1,8,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,0,1,1,0,0.14288154244422913
60,0,860,12,5,318,1,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,1,0,0.19637687504291534
54,1,0,18,5,17,11,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,1,0,1,0,0.0006146965897642076
31,0,213,20,5,74,2,-1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,1,0,

Congratulations! you have successfully deployed the trained machine learning model using Amazon SageMaker to get predictions.

### Cleanup

You have completed this notebook. To move to the next part of the lab, do the following:

- Close this notebook file.
- Return to the lab session and continue with the **Conclusion**.

## Additional resources

- [Deploy models for inference](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-model.html)
- [Using SageMaker APIs](https://sagemaker.readthedocs.io/en/stable/api/inference/index.html)

<a name="appendix" id="appendix"></a>

## Appendix


<a name="task3-2-1" id="task3-2-1"></a>

### Upload model and test data to S3 (Task 3.2)

In [ ]:
# Upload the model to the Amazon S3 bucket
from sagemaker.session import Session
from sagemaker.image_uris import retrieve

#session = boto3.Session()
sagemaker = boto3.client('sagemaker')
s3_client = boto3.client("s3")

#upload-model
s3_client.upload_file(Filename="model.tar.gz", Bucket=bucket, Key=f"{prefix}/models/model.tar.gz")  
model_path = f"s3://{bucket}/{prefix}/models/model.tar.gz"
print(model_path)

#upload-dataset
s3_client.upload_file(Filename="test_data.csv", Bucket=bucket, Key=f"{prefix}/test_data.csv", ExtraArgs={"ContentType": "text/csv;charset=utf-8"})
batch_path = f"s3://{bucket}/{prefix}/test_data.csv"
print(batch_path)

To continue this lab, return to <a href="#task3-2-continue" target="_self">Task 3.2</a>.

<a name="task3-2-2" id="task3-2-2"></a>

### Create a model in SageMaker (Task 3.2)

In [ ]:
sagemaker = boto3.client('sagemaker')

# Retrieve the container imagez
image = retrieve(framework='xgboost', region=boto3.Session().region_name, version='1.5-1')

primary_container = {"Image": image, "ModelDataUrl": model_path}

# Set up the model
create_model_response = sagemaker.create_model(
    ModelName='telemarketing-model', ExecutionRoleArn=role, PrimaryContainer=primary_container
)

print(create_model_response["ModelArn"])

To continue this lab, return to <a href="#task3-3" target="_self">Task 3.3</a>.

<a name="task3-3-1" id="task3-3-1"></a>

### Create a Batch Transformer Object (Task 3.3)

In [ ]:
from sagemaker.transformer import Transformer

#create-batch-transformer
transformer = Transformer(
    model_name="telemarketing-model",
    instance_type="ml.m4.xlarge",
    instance_count=1,
    assemble_with="Line",
    strategy="MultiRecord",
    accept="text/csv",
    output_path="s3://{}/{}/batch-transform/test".format(bucket, prefix)
)

To continue this lab, return to <a href="#task3-4" target="_self">Task 3.4</a>.

<a name="task3-4-1" id="task3-4-1"></a>

### Run a batch transform job (Task 3.4)


In [ ]:
#run-batch-transform-job
transformer.transform(batch_path, content_type="text/csv", split_type="Line", join_source="Input")

transformer.wait()

To continue this lab, return to <a href="#task3-5" target="_self">Task 3.5</a>.